
# Classification Decision Tree
We will implement a Classification Decision Tree from scratch in the following problem. Then we will use our model to predict malignant and benign breast cancer. For this purpose, we will use the breast_cancer.csv dataset which you can find more details about it <a href="https://www.kaggle.com/datasets/merishnasuwal/breast-cancer-prediction-dataset"><font face="Roboto">here</font></a>.

In [165]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import log
import random


## Classification Decision Tree Class


In [166]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        if self.value is not None:
            return True
        return False

In [167]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
        self.nodes=[]

    def is_splitting_finished(self, depth, num_class_labels, num_samples):
        if(depth>=self.max_depth or num_class_labels==1 or num_samples<self.min_samples_split):return True

        return False

    def split(self, X, y, feature, threshold):
        left_indexes = X[feature] <= threshold
        right_indexes = -left_indexes
        X_left = X[left_indexes]
        y_left = y[left_indexes]
        X_right = X[right_indexes]
        y_right = y[right_indexes]

        return X_left, X_right, y_left, y_right

    def entropy(self, y):
        import numpy as np
        ent=0
    
    
        P=y.value_counts()/len(y)

        for p in P  :
            if p>0 :
                 ent+=p*log(p)
        ent=-ent
        return ent


    def information_gain(self, X, y, feature, threshold):
        X_left, X_right, y_left, y_right = self.split (X,y,feature,threshold)

        if(len(y_right)==0 or len(y_left)==0):return 0
        H_y = self.entropy(y)

        p = len(y_left)/len(y)*self.entropy(y_left)+len(y_right)/len(y)*self.entropy(y_right)
        ig=H_y-p
        return ig

    def best_split(self, X, y):
        best_ig=0
        features=X.columns
        best_f=features[0]
        best_t=0
        for f in features:
            tresholds=X[f].unique()
            for t in tresholds:
                ig=self.information_gain(X,y,f,t)

                if(ig>best_ig):
                    best_ig=ig
                    best_f=f
                    best_t=t

        return best_f, best_t

    def build_tree(self, X, y, depth=0):
        if self.is_splitting_finished(depth, len(X.columns), len(X)):
            X.columns
            return None

        best_feature, best_threshold = self.best_split(X, y)

        X_left, X_right, y_left, y_right = self.split(X, y, best_feature, best_threshold)

        left_node = self.build_tree(X_left, y_left, depth=depth + 1)
        right_node = self.build_tree(X_right, y_right, depth=depth + 1)

        value = None
        if left_node is None or right_node is None:
            true_value = len(y[y['diagnosis'] == 1])
            false_value = len(y[y['diagnosis'] == 0])
            if true_value >= false_value:
                value = 1
            else:
                value = 0
        cc=Node(feature=best_feature, threshold=best_threshold, left=left_node, right=right_node, value=value)
        self.nodes.append(cc)
        return cc

    def fit(self, X, y):
        self.root = self.build_tree(X, y)

    def predict(self, X):
        tree = self.root
        predicted_value = []
        for index in list(X.index):
            data = X.loc[index]
            current_tree = tree
            for depth in range(self.max_depth):
                if Node.is_leaf(current_tree):
                    predicted_value.append(current_tree.value)
                    break
                feature = current_tree.feature
                threshold = current_tree.threshold
                if data[feature] <= threshold:
                    current_tree = current_tree.left
                if data[feature] > threshold:
                    current_tree = current_tree.right

        return predicted_value

## Data Prepration
We performd a good EDA for data. Then split it into train and validation data. We will then use the validation data to find the best model hyperparameters.  

In [168]:
breast_cancer_pdf = pd.read_csv("breast_cancer.csv")
breast_cancer_pdf.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


In [169]:
# TODO

In [170]:

X = breast_cancer_pdf.drop(['diagnosis'],axis=1)
import numpy as np
y = breast_cancer_pdf[['diagnosis']]
print(np.bincount(y['diagnosis']))
print(y.value_counts()/len(y))

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.70, random_state=42)

[212 357]
diagnosis
1            0.627417
0            0.372583
dtype: float64


In [171]:
y_val.head()

,diagnosis
204,1
70,0
131,0
431,1
540,1



## Training And Tuning Hyperparameters
We need to find the best hyperparameters for our model. Your model must have at least accuracy=0.85 on validation data.

In [172]:
max_depths = [5,6,10,11,12,13]
min_samples_splits = [2,3,4,5]

In [173]:
best_max_depth = None
best_min_samples_split = None
best_accuracy = 0
best_model = None
for max_depth in max_depths:
    for min_samples_split in min_samples_splits:
        clf = DecisionTree(max_depth, min_samples_split)
        clf.fit(x_train, y_train)
        y_val_pred = clf.predict(x_val)
        y_train_pred = clf.predict(x_train)
        accuracy = accuracy_score(y_val_pred, y_val)
        train_accuracy = accuracy_score(y_train_pred, y_train)
        print(f"accuracy of training set for [min_samples_splits={min_samples_split}-max_depths={max_depth}] ={train_accuracy}")
        print(f"accuracy of validation set for [min_samples_splits={min_samples_split}-max_depths={max_depth}] ={accuracy}")
        print("------------------------------------------------")
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_max_depth = max_depth
            best_min_samples_split = min_samples_split
            best_model = clf
            # for node in clf.nodes:
            #     print(node.feature)
            #     print(node.threshold)
            #     print(node.value)

accuracy of training set for [min_samples_splits=2-max_depths=5] =0.9294117647058824
accuracy of validation set for [min_samples_splits=2-max_depths=5] =0.9047619047619048
------------------------------------------------
accuracy of training set for [min_samples_splits=3-max_depths=5] =0.9294117647058824
accuracy of validation set for [min_samples_splits=3-max_depths=5] =0.9047619047619048
------------------------------------------------
accuracy of training set for [min_samples_splits=4-max_depths=5] =0.9294117647058824
accuracy of validation set for [min_samples_splits=4-max_depths=5] =0.9047619047619048
------------------------------------------------
accuracy of training set for [min_samples_splits=5-max_depths=5] =0.9294117647058824
accuracy of validation set for [min_samples_splits=5-max_depths=5] =0.9047619047619048
------------------------------------------------
accuracy of training set for [min_samples_splits=2-max_depths=6] =0.9588235294117647
accuracy of validation set for 